# Deploy a BigQuery ML user churn propensity model to Vertex AI for online predictions

In [ ]:
!pip3 install google-cloud-aiplatform --user
!pip3 install pyarrow==11.0.0 --user
!pip3 install --upgrade google-cloud-bigquery --user
!pip3 install --upgrade google-cloud-bigquery-storage --user
!pip3 install --upgrade google-cloud-storage --user
!pip install db-dtypes

**Restart the kernel and ignore the compatibility errors.**

### Define constants

In [ ]:
# Retrieve and set PROJECT_ID and REGION environment variables.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

**Note:** Replace the <code>REGION</code> with the associated region mentioned in the qwiklabs resource panel.

In [ ]:
BQ_LOCATION = 'US'
REGION = 'us-central1'

### Import libraries

In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd

### Create a GCS bucket for artifact storage

Create a globally unique Google Cloud Storage bucket for artifact storage. You will use this bucket to export your BQML model later in the lab and upload it to Vertex AI.

In [ ]:
GCS_BUCKET = f"{PROJECT_ID}-bqmlga4"

In [ ]:
!gsutil mb -l $REGION gs://$GCS_BUCKET

### Create a BigQuery dataset

Next, create a BigQuery dataset from this notebook using the Python-based [`bq` command line utility](https://cloud.google.com/bigquery/docs/bq-command-line-tool). 

This dataset will group your feature views, model, and predictions table together. You can view it in the [BigQuery](https://pantheon.corp.google.com/bigquery) console.

In [ ]:
BQ_DATASET = f"{PROJECT_ID}:bqmlga4"

In [ ]:
!bq mk --location={BQ_LOCATION} --dataset {BQ_DATASET}

### Initialize the Vertex Python SDK client

Import the Vertex SDK for Python into your Python environment and initialize it.

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{GCS_BUCKET}")

## Exploratory Data Analysis (EDA) in BigQuery

This lab uses a [public BigQuery dataset]() that contains raw event data from a real mobile gaming app called **Flood it!** ([Android app](https://play.google.com/store/apps/details?id=com.labpixies.flood), [iOS app](https://itunes.apple.com/us/app/flood-it!/id476943146?mt=8)).

The data schema originates from Google Analytics for Firebase but is the same schema as [Google Analytics 4](https://support.google.com/analytics/answer/9358801).

Take a look at a sample of the raw event dataset using the query below:

In [ ]:
MODEL_NAME="churn_xgb"

In [38]:
%%bigquery --project $PROJECT_ID
CREATE OR REPLACE MODEL `bqmlga4.churn_xgb`
  OPTIONS (model_type="logistic_reg",
      max_iterations=3, input_label_cols=["species"],num_trials=1)
  AS SELECT
    *
  FROM
    `bigquery-public-data.ml_datasets.iris`;

Query is running:   0%|          |

""


In [39]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_xgb)
WHERE trial_id=1;

Query is running:   0%|          |

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Downloading:   0%|          |

,trial_id,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1,0.865079,0.826667,0.826667,0.817851,0.544638,0.96412


In [40]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bigquery-public-data.ml_datasets.iris))

Query is running:   0%|          |

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Downloading:   0%|          |

,trial_id,predicted_species,predicted_species_probs,sepal_length,sepal_width,petal_length,petal_width,species
0,1,versicolor,"[{'label': 'versicolor', 'prob': 0.47182750084...",6.3,2.3,4.4,1.3,versicolor
1,1,virginica,"[{'label': 'virginica', 'prob': 0.744802300746...",7.7,2.8,6.7,2.0,virginica
2,1,virginica,"[{'label': 'virginica', 'prob': 0.495413042001...",6.5,2.8,4.6,1.5,versicolor
3,1,virginica,"[{'label': 'virginica', 'prob': 0.497731373463...",5.9,3.2,4.8,1.8,versicolor
4,1,virginica,"[{'label': 'virginica', 'prob': 0.553471503524...",7.0,3.2,4.7,1.4,versicolor
...,...,...,...,...,...,...,...,...
145,1,virginica,"[{'label': 'virginica', 'prob': 0.758622856394...",7.6,3.0,6.6,2.1,virginica
146,1,versicolor,"[{'label': 'versicolor', 'prob': 0.43353099808...",5.8,2.6,4.0,1.2,versicolor
147,1,setosa,"[{'label': 'setosa', 'prob': 0.839997942681406...",5.1,3.8,1.9,0.4,setosa
148,1,setosa,"[{'label': 'setosa', 'prob': 0.824576405856592...",4.7,3.2,1.3,0.2,setosa


In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  # Declare label column.
  INPUT_LABEL_COLS=["variety"],
  # Specify custom data splitting using the `data_split` column.
  DATA_SPLIT_METHOD="AUTO_SPLIT",
  # Enable Vertex Explainable AI aggregated feature attributions.
  ENABLE_GLOBAL_EXPLAIN=false,
  # Hyperparameter tuning arguments.
  num_trials=1,
  max_parallel_trials=5,
  HPARAM_TUNING_OBJECTIVES=["F1_SCORE"],
  EARLY_STOP=True,
  # Hyperpameter search space.
  LEARN_RATE=0,
  MAX_TREE_DEPTH=1
) AS

SELECT
  *
FROM
  bqmlga4.iris

## Evaluate BQML XGBoost model performance

Once training is finished, you can run [ML.EVALUATE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) to return model evaluation metrics. By default, all model trials will be returned so the below query just returns the model performance for optimal first trial.

In [39]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_xgb)
WHERE trial_id=1;

Query is running:   0%|          |

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Downloading:   0%|          |

,trial_id,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1,0.865079,0.826667,0.826667,0.817851,0.544638,0.96412


ML.EVALUATE generates the [precision, recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall), [accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy), [log_loss](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss), [f1_score](https://en.wikipedia.org/wiki/F-score) and [roc_auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc) using the default classification threshold of 0.5, which can be modified by using the optional `THRESHOLD` parameter.

Next, use the [ML.CONFUSION_MATRIX](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-confusion) function to return a confusion matrix for the input classification model and input data.

For more information on confusion matrices, you can read through a detailed explanation [here](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative).

You can also plot the AUC-ROC curve by using [ML.ROC_CURVE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-roc) to return the metrics for different threshold values for the model.

## Inspect global feature attributions

To provide further context to your model performance, you can use the [ML.GLOBAL_EXPLAIN](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-global-explain#get_global_feature_importance_for_each_class_of_a_boosted_tree_classifier_model) function which leverages Vertex Explainable AI as a back-end. [Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai) helps you understand your model's outputs for classification and regression tasks. Specifically, Vertex AI tells you how much each feature in the data contributed to your model's predicted result. You can then use this information to verify that the model is behaving as expected, identify and mitigate biases in your models, and get ideas for ways to improve your model and your training data.

## Generate batch predictions

You can generate batch predictions for your BQML XGBoost model using [ML.PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict).

In [40]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bigquery-public-data.ml_datasets.iris))

Query is running:   0%|          |

/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/opt/conda/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


Downloading:   0%|          |

,trial_id,predicted_species,predicted_species_probs,sepal_length,sepal_width,petal_length,petal_width,species
0,1,versicolor,"[{'label': 'versicolor', 'prob': 0.47182750084...",6.3,2.3,4.4,1.3,versicolor
1,1,virginica,"[{'label': 'virginica', 'prob': 0.744802300746...",7.7,2.8,6.7,2.0,virginica
2,1,virginica,"[{'label': 'virginica', 'prob': 0.495413042001...",6.5,2.8,4.6,1.5,versicolor
3,1,virginica,"[{'label': 'virginica', 'prob': 0.497731373463...",5.9,3.2,4.8,1.8,versicolor
4,1,virginica,"[{'label': 'virginica', 'prob': 0.553471503524...",7.0,3.2,4.7,1.4,versicolor
...,...,...,...,...,...,...,...,...
145,1,virginica,"[{'label': 'virginica', 'prob': 0.758622856394...",7.6,3.0,6.6,2.1,virginica
146,1,versicolor,"[{'label': 'versicolor', 'prob': 0.43353099808...",5.8,2.6,4.0,1.2,versicolor
147,1,setosa,"[{'label': 'setosa', 'prob': 0.839997942681406...",5.1,3.8,1.9,0.4,setosa
148,1,setosa,"[{'label': 'setosa', 'prob': 0.824576405856592...",4.7,3.2,1.3,0.2,setosa


The following query returns the probability that the user will return after 24 hrs. The higher the probability and closer it is to 1, the more likely the user is predicted to churn, and the closer it is to 0, the more likely the user is predicted to return.

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE TABLE bqmlga4.churn_predictions AS (
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.ml_features))
);